## Collect video sample from internet

video source: 

- Free City Street Footage - Royalty Free Stock Footage - People Walking Stock Footage No Copyright

  https://www.youtube.com/watch?v=bwJ-TNu0hGM


tools: 
- Video Downloader

  https://x2convert.com/



## Read video frames

https://theailearner.com/2018/10/15/extracting-and-saving-video-frames-using-opencv-python/

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
%cd content

In [ ]:

video_file_path = './city.mp4'
frames = []

cap= cv2.VideoCapture(video_file_path)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    frames.append(frame)
cap.release()

frames = np.array(frames)


In [ ]:
frames.shape

In [ ]:
cv2_imshow(frames[0,:,:,:])

## Object Detection


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -xvf ./ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz


In [ ]:
import tensorflow as tf
import time

model_path = '/content/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model'

start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load(model_path)
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

In [ ]:
process_size = 360
score_threshold = 0.6

process_num = frames.shape[0]

if process_num > 2000:
  process_num = 2000

conseq_bbs = []
min_size = {
    'width': 0.01,
    'height': 0.02
}

for i in range(process_num):
  frame = frames[i,:,:,:3]
  original_size = frame.shape
  frame_height, frame_width = original_size[:2]

  cur_frame = cv2.resize(frame,(process_size, process_size))

  input_image = cur_frame.reshape(1, cur_frame.shape[0], cur_frame.shape[1], 3)

  detections = detect_fn(input_image)

  num_detections = int(detections['num_detections'].numpy())
  obj_bbs = []
  obj_ids = []
  for j in range(num_detections):
    class_id = int(detections['detection_classes'][0][j].numpy())
    score = float(detections['detection_scores'][0][j].numpy())
    bbox = [float(v) for v in detections['detection_boxes'][0][j].numpy()]

    x = bbox[1] * frame_width
    y_ = bbox[0] * frame_height
    right = bbox[3] * frame_width
    bottom = bbox[2] * frame_height

    bbox = [x, y_, right, bottom]
    obj_width = bbox[3]-bbox[1]
    obj_height = bbox[2]-bbox[0]
    if score > score_threshold and class_id == 1 and obj_width >= frame_width*min_size['width']:
      obj_bbs.append(bbox)
      obj_ids.append(class_id)

  conseq_bbs.append(obj_bbs)

  if i%100 == 0:
    print(i, time.time())

In [ ]:
len(conseq_bbs)

## Draw Bounding Boxes

In [ ]:
frame_idx = 0

obj_bbs_tmp = conseq_bbs[frame_idx]

frame_copy = frames[frame_idx, :,:,:].copy()
frame_height, frame_width = original_size[:2]

for i in range(len(obj_bbs_tmp)):

  x = obj_bbs_tmp[i][1] * frame_width
  y_ = obj_bbs_tmp[i][0] * frame_height
  right = obj_bbs_tmp[i][3] * frame_width
  bottom = obj_bbs_tmp[i][2] * frame_height
  
  cv2.rectangle(frame_copy, (int(x), int(y_)), (int(right), int(bottom)), (0,0,255), thickness=2)


In [ ]:
if len(obj_bbs_tmp) > 0:
  obj_bbs_tmp[frame_idx]

In [ ]:
cv2_imshow(frame_copy)

## Write video to a file

In [ ]:
# frame_height, frame_width = original_size[:2]
# out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# total_frame_number = len(conseq_bbs)
# print('total frame number:', total_frame_number)

# for idx in range(total_frame_number):
#   frame_copy = frames[idx].copy()
#   obj_bbs_tmp = conseq_bbs[idx]

#   for i in range(len(obj_bbs_tmp)):

#     x = obj_bbs_tmp[i][1] * frame_width
#     y_ = obj_bbs_tmp[i][0] * frame_height
#     right = obj_bbs_tmp[i][3] * frame_width
#     bottom = obj_bbs_tmp[i][2] * frame_height

#     obj_bbs_tmp[i] = [x, y_, right, bottom]
    
#     cv2.rectangle(frame_copy, (int(x), int(y_)), (int(right), int(bottom)), (0,0,255), thickness=2)
#   out.write(frame_copy)
# out.release()

## Object tracking

In [ ]:
# https://www.pyimagesearch.com/2018/07/23/simple-object-tracking-with-opencv/

# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
import random

class CentroidTracker():
	def __init__(self, maxDisappeared=50, maxDistance=50):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.boxes = OrderedDict()
		self.colors = OrderedDict()
		self.histories = OrderedDict()
		self.disappeared = OrderedDict()
	
		self.maxdist = maxDistance

		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

	def register(self, centroid, box):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.boxes[self.nextObjectID] = box
		self.histories[self.nextObjectID] = [(int((box[0]+box[2])/2), int(box[3]))]
		self.disappeared[self.nextObjectID] = 0
		r = random.randint(0,255)
		self.colors[self.nextObjectID] = (r, 
		                                	random.randint(0,255), 
																			255-r)
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.boxes[objectID]
		del self.disappeared[objectID]
		# keep histories
		# del self.histories[objectID]
		del self.colors[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			deregister_list = []
			for objectID in self.disappeared.keys():
				self.disappeared[objectID] += 1

				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					deregister_list.append(objectID)

			if len(deregister_list) > 0:
				for obj_id in deregister_list:
					self.deregister(obj_id)

			# return early as there are no centroids or tracking info
			# to update
			return self.objects, self.boxes, self.colors, self.histories

		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")
		inputBoxes = np.zeros((len(rects), 4), dtype="int")

		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)
			inputBoxes[i] = (startX, startY, endX, endY)

		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i], inputBoxes[i])

		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())
			# objectBoxes = list(self.boxes.values())

			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)

			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value as at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()

			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]

			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()

			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue

				if D[row, col] > self.maxdist:
					continue

				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.boxes[objectID] = inputBoxes[col]
				self.histories[objectID].append((int((inputBoxes[col][0]+inputBoxes[col][2])/2), int(inputBoxes[col][3])))
				self.disappeared[objectID] = 0

				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)

			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1

					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)

			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col], inputBoxes[col])

		# return the set of trackable objects
		return self.objects, self.boxes, self.colors, self.histories

In [ ]:
ct = CentroidTracker(maxDisappeared=10, maxDistance=20)

frame_height, frame_width = original_size[:2]
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 20, (frame_width,frame_height))

total_frame_number = len(conseq_bbs)
print('total frame number:', total_frame_number)

for idx in range(total_frame_number):
  frame_copy = frames[idx].copy()
  rects = conseq_bbs[idx]
  
  (objects, boxes, colors, histories) = ct.update(rects)
  # print(idx, rects)

	# loop over the tracked objects
  for (i, (objectID, centroid)) in enumerate(objects.items()):
		# draw both the ID of the object and the centroid of the
		# object on the output frame
    text = str(objectID)
    cv2.putText(frame_copy, text, (centroid[0] - 10, centroid[1] - 10),
      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    x = boxes[objectID][0]
    y_ = boxes[objectID][1]
    right = boxes[objectID][2]
    bottom = boxes[objectID][3]

    cv2.rectangle(frame_copy, (int(x), int(y_)), (int(right), int(bottom)), colors[objectID], thickness=2)
    
  out.write(frame_copy)
out.release()


## Background estimation
https://www.learnopencv.com/simple-background-estimation-in-videos-using-opencv-c-python/


In [ ]:
background = np.median(frames[:], axis=0).astype(dtype=np.uint8)

In [ ]:
cv2_imshow(background)

## Trajectory history

In [ ]:
background_copy = background.copy()
line_thickness = 5
# line_thickness_max = 10
min_history = 2
history_list = []

for key in histories.keys():
  history = histories[key]
  if len(history) > min_history:
    for i in range(len(history)-1):
      cv2.line(background_copy, (history[i][0], history[i][1]), (history[i+1][0], history[i+1][1]), (0, 0, 255), thickness=line_thickness)


In [ ]:
cv2_imshow(background_copy)

## Trajectory density

In [ ]:
print('frame height:', frame_height, 'frame width:', frame_width)

In [ ]:
x_range = 17
y_range = 9

xedges = [i*40 for i in range(x_range)]
yedges = [i*40 for i in range(y_range)]

history_x = []
history_y = []

for key in histories.keys():
  if len(histories[key]) > min_history:
    for h in histories[key]:
      history_x.append(h[0])
      history_y.append(h[1])

H, xedges, yedges = np.histogram2d(history_x, history_y, bins=(xedges, yedges))
H = np.flipud(H.T)
H

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.imshow(H, interpolation='nearest', origin='low',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

## Apply Trajectory Density to Image

In [ ]:
import math

background_density = background.copy()
density_mult = 5
darken_by = 50
box_margin = 0

H_ = H.T

block_size = (40, 40)

max_count = np.max(np.max(H))
print('max count:', max_count)

for i in range(x_range-1):
  for j in range(y_range-1):
    if H_[i,y_range-j-2] > 0:
      background_density[(j*block_size[1]):((j+1)*block_size[1]-box_margin), (i*block_size[0]):((i+1)*block_size[0]-box_margin), :] = np.clip(np.int16(background_density[(j*block_size[1]):((j+1)*block_size[1]-box_margin), (i*block_size[0]):((i+1)*block_size[0]-box_margin), :]) + (0,0,density_mult*H_[i,y_range-j-2]*255/max_count), 0,255)
    else:
      background_density[(j*block_size[1]):((j+1)*block_size[1]-box_margin), (i*block_size[0]):((i+1)*block_size[0]-box_margin), :] = np.clip(np.int16(background_density[(j*block_size[1]):((j+1)*block_size[1]-box_margin), (i*block_size[0]):((i+1)*block_size[0]-box_margin), :]) - (darken_by), 0,255)
      

In [ ]:
cv2_imshow(background_density)

## Copy the video to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cp /content/output.mp4 /content/drive/My\ Drive/

## Analysis
- People are mostly walking on the left lane and cars are using the right lane. That's the reason why trajectory density is lower on the right lane.

- Small area on the lower left has low trajectory density because there is a bike and a tree. People have to walk a bit further to get onto the sidewalk.

- Some people are undetected by the model. Trying different models for human detection should help this.

- Some different people are tracked as the same person. More complex tracking algorithm, such as color features, should help this.
